In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activations):
        super(MultiLayerPerceptron, self).__init__()
        assert len(hidden_sizes) == len(activations), "Number of hidden layers should match number of activation functions"
        layers = []
        prev_size = input_size
        # Construct the layers of the MLP
        for i in range(len(hidden_sizes)):
            layers.append(nn.Linear(prev_size, hidden_sizes[i]))
            layers.append(activations[i])
            prev_size = hidden_sizes[i]
        layers.append(nn.Linear(prev_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Define the input size, hidden layer sizes, output size, and activation functions
input_size = 28 * 28
hidden_sizes = [256, 128]
output_size = 10
activations = [nn.ReLU(), nn.ReLU()]

# Create an instance of the MLP model
model = MultiLayerPerceptron(input_size, hidden_sizes, output_size, activations)

# Define transformations for preprocessing the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset and apply the transformations
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

# Create data loaders for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test dataset
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on test images: {:.2f}%'.format(100 * correct / total))



100%|██████████| 9912422/9912422 [00:00<00:00, 80213831.55it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 23192742.45it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 32567133.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18040273.45it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 0.2327
Epoch [2/5], Loss: 0.4216
Epoch [3/5], Loss: 0.1193
Epoch [4/5], Loss: 0.0410
Epoch [5/5], Loss: 0.0401
Accuracy on test images: 96.62%
